# CONCLUSION



import et ouverture du fichier

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

import plotly.express as px
import plotly.graph_objects as go



try:
    df = pd.read_csv('dvf_loyers.csv', sep=';')
except FileNotFoundError:
    print("Erreur: Le fichier 'dvf_loyers.csv' est introuvable.")
    print("Veuillez vous assurer que le fichier est dans le même répertoire que votre notebook.")
    # Crée un DataFrame vide pour éviter d'autres erreurs si le fichier n'est pas trouvé
    df = pd.DataFrame(columns=['valeur_fonciere', 'surface_reelle_bati', 'loyer_ref_m2', 'adresse_nom_voie', 'nom_commune', 'loyer_majore_m2', 'loyer_minore_m2'])

Nettoyer les données

In [ ]:
df = df.dropna(subset=['valeur_fonciere', 'surface_reelle_bati', 'loyer_ref_m2', 'loyer_majore_m2', 'loyer_minore_m2'])
df = df[df['surface_reelle_bati'] > 0]
df = df[df['loyer_ref_m2'] > 0]

Créer la liste des biens pour le menu déroulant

In [ ]:
df['bien_label'] = df.apply(
    lambda row: f"{row['adresse_nom_voie']}, {row['nom_commune']} - {row['valeur_fonciere']:,.0f}€ - {row['surface_reelle_bati']:.0f}m²",
    axis=1
)
quartiers = ["Tous"] + sorted(df["nom_commune"].unique().tolist())

Widgets de paramètres

In [ ]:
quartier_selector = widgets.Dropdown(
    options=quartiers,
    value="Tous",
    description="Quartier :",
    style={'description_width': '150px'},
    layout=widgets.Layout(width='90%')
)
bien_selector = widgets.Dropdown(
    options=list(zip(df['bien_label'], df.index)),
    description='Bien:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='90%')
)

apport_slider = widgets.IntSlider(
    value=50000,
    min=0,
    max=500000,  # Cette valeur max sera maintenant dynamique
    step=5000,
    description='Apport (€):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='80%')
)

taux_slider = widgets.FloatSlider(
    value=3.5,
    min=0.0,
    max=15.0,
    step=0.1,
    description='Taux prêt (%):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='80%')
)

duree_slider = widgets.IntSlider(
    value=25,
    min=5,
    max=30,
    step=1,
    description='Durée (années):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='80%')
)

output = widgets.Output()

fonction de calcule des mensualité banquaire

In [ ]:
def calculer_mensualite(capital, taux_annuel, duree_annees):
    """Calcule la mensualité d'un prêt"""
    if capital <= 0:
        return 0
    taux_mensuel = taux_annuel / 100 / 12
    nb_mois = duree_annees * 12
    if taux_mensuel == 0:
        return capital / nb_mois
    mensualite = capital * (taux_mensuel * (1 + taux_mensuel)**nb_mois) / ((1 + taux_mensuel)**nb_mois - 1)
    return mensualite

fonction de filtrage en fonction du quartier

In [ ]:
def mettre_a_jour_biens(change=None):
    """Met à jour la liste des biens selon le quartier sélectionné"""
    selected_quartier = quartier_selector.value
    
    if selected_quartier == "Tous":
        df_filtre = df
    else:
        df_filtre = df[df["nom_commune"] == selected_quartier]
    
    # Met à jour le dropdown des biens
    bien_selector.options = list(zip(df_filtre['bien_label'], df_filtre.index))
    
    # Sélectionne le premier bien du filtre si dispo
    if len(df_filtre) > 0:
        bien_selector.value = df_filtre.index[0]


fonction d'affichage

In [ ]:
def afficher_resultats(change=None):
    """Affiche les résultats de la simulation"""
    with output:
        clear_output(wait=True)
        
        # S'assurer qu'il y a des données à traiter
        if df.empty or bien_selector.value is None:
            print("Aucune donnée à afficher. Veuillez vérifier le chargement du fichier CSV.")
            return
            
        # Récupérer le bien sélectionné
        idx = bien_selector.value
        bien = df.loc[idx]
        
        # Paramètres de financement
        prix_achat = bien['valeur_fonciere']

        # --- MODIFICATION ---
        # 1. Mettre à jour la valeur maximale du curseur d'apport avec le prix du bien
        apport_slider.max = prix_achat
        
        # 2. Si l'apport actuel est supérieur au prix du bien, le ramener au maximum possible
        if apport_slider.value > prix_achat:
            apport_slider.value = prix_achat
        # --- FIN DE LA MODIFICATION ---
            
        apport = apport_slider.value
        taux = taux_slider.value
        duree = duree_slider.value
        
        # Calculs
        montant_emprunte = max(0, prix_achat - apport)
        mensualite_pret = calculer_mensualite(montant_emprunte, taux, duree)
        cout_total_credit = mensualite_pret * duree * 12
        cout_total_projet = cout_total_credit + apport
        
        # Calculs des loyers
        surface = bien['surface_reelle_bati']
        loyer_estime = surface * bien['loyer_ref_m2']
        loyer_majore = surface * bien['loyer_majore_m2']
        loyer_minore = surface * bien['loyer_minore_m2']
        
        # Charges estimées (25% du loyer)
        charges_estimees = loyer_estime * 0.25
        
        # Cash-flow
        cashflow_net = loyer_estime - mensualite_pret
        cashflow_max = loyer_majore - mensualite_pret
        
        # Rendements
        rendement_brut = (loyer_estime * 12 / prix_achat) * 100 if prix_achat > 0 else 0
        rendement_locatif_majore = (loyer_majore * 12 / prix_achat) * 100 if prix_achat > 0 else 0
        
        # Affichage formaté
        print("\n" + "-"*80)
        print("FINANCEMENT")
        print("-"*80)
        apport_pourcentage = (apport / prix_achat * 100) if prix_achat > 0 else 0
        print(f"Apport personnel:     {apport:>15,.0f} € ({apport_pourcentage:.1f}%)")
        print(f"Montant emprunté:     {montant_emprunte:>15,.0f} €")
        print(f"Mensualité du prêt:   {mensualite_pret:>15,.0f} €/mois")
        print(f"Coût total du crédit: {cout_total_credit:>15,.0f} €")
        print(f"Coût total du projet: {cout_total_projet:>15,.0f} € (Apport + Coût crédit)")


        print("\n" + "-"*80)
        print("LOYERS ET RENTABILITÉ")
        print("-"*80)
        print(f"Loyer MINORÉ:         {loyer_minore:>15,.0f} €/mois")
        print(f"Loyer de RÉFÉRENCE:   {loyer_estime:>15,.0f} €/mois")
        print(f"Loyer MAJORÉ:         {loyer_majore:>15,.0f} €/mois\n")

        print(f"Loyer annuel (réf.):  {loyer_estime*12:>15,.0f} €")
        print(f"Loyer/m² (réf.):      {bien['loyer_ref_m2']:>15.2f} €")
        print(f"Charges locatives:    {charges_estimees:>15,.0f} €/mois (estimé à 25%)")
        print(f"(refacturées au locataire)")
        
        cashflow_status_net = "POSITIF" if cashflow_net >= 0 else "NÉGATIF"
        print(f"\nCash-flow net (réf.): {cashflow_net:>15,.0f} €/mois ({cashflow_status_net})")
        cashflow_status_max = "POSITIF" if cashflow_max >= 0 else "NÉGATIF"
        print(f"Cash-flow max (majoré):{cashflow_max:>15,.0f} €/mois ({cashflow_status_max})")
        
        print(f"\nRendement brut (réf.):     {rendement_brut:>12.2f} %")
        print(f"Rendement brut (majoré):   {rendement_locatif_majore:>12.2f} %")


In [ ]:
# Observer les changements
quartier_selector.observe(mettre_a_jour_biens, names='value')
bien_selector.observe(afficher_resultats, names='value')
apport_slider.observe(afficher_resultats, names='value')
taux_slider.observe(afficher_resultats, names='value')
duree_slider.observe(afficher_resultats, names='value')

affichage

In [ ]:
# Affichage des widgets
display(quartier_selector)
display(bien_selector)
display(apport_slider)
display(taux_slider)
display(duree_slider)
display(output)

# Affichage initial
afficher_resultats()

Dropdown(description='Quartier :', layout=Layout(width='90%'), options=('Tous', 'Paris 10e Arrondissement', 'P…

Dropdown(description='Bien:', layout=Layout(width='90%'), options=(('PAS BASFOUR, Paris 2e Arrondissement - 17…

IntSlider(value=50000, description='Apport (€):', layout=Layout(width='80%'), max=500000, step=5000, style=Sli…

FloatSlider(value=3.5, description='Taux prêt (%):', layout=Layout(width='80%'), max=15.0, style=SliderStyle(d…

IntSlider(value=25, description='Durée (années):', layout=Layout(width='80%'), max=30, min=5, style=SliderStyl…

Output()

recuperation des données pour la partie carte

In [ ]:
for col in ['longitude','latitude','valeur_fonciere','surface_reelle_bati','loyer_ref_m2']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.dropna(subset=['latitude','longitude','surface_reelle_bati','loyer_ref_m2','valeur_fonciere'], inplace=True)

df['Loyer_Mensuel_Estime'] = df['surface_reelle_bati'] * df['loyer_ref_m2']

ajout du filtre budget

In [ ]:


budget_slider = widgets.IntSlider(
    value=300000, min=0, max=1_000_000, step=10_000,
    description='Budget (€):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='80%')
)


In [ ]:
# ==================== Zones ====================
calc_output = widgets.Output()
click_output = widgets.Output()
map_output = widgets.Output()

fonction pour connaitre ses mensualité maximun

In [ ]:
# ==================== Fonctions ====================
def mensualite_vect(capital, taux_annuel, duree_annees):
    capital = np.asarray(capital, dtype=float)
    n = int(duree_annees) * 12
    tm = (taux_annuel / 100.0) / 12.0
    if tm == 0:
        return np.where(capital > 0, capital / n, 0.0)
    num = tm * (1 + tm)**n
    den = (1 + tm)**n - 1
    return np.where(capital > 0, capital * (num / den), 0.0)

fonction de rendement, affichage en couleur en fonction du rendement du bien

In [ ]:
def render(change=None):
    budget = budget_slider.value
    apport  = apport_slider.value
    taux    = taux_slider.value
    duree   = duree_slider.value

    dff = df[df['valeur_fonciere'] <= budget].copy()
    if dff.empty:
        with calc_output:
            clear_output(wait=True)
            print(f"Aucun bien ≤ {budget:,.0f} €.")
        with map_output:
            clear_output(wait=True)
            print("Aucune donnée à afficher.")
        return

    dff['Capital_Emprunte'] = (dff['valeur_fonciere'] - apport).clip(lower=0)
    dff['Mensualite'] = mensualite_vect(dff['Capital_Emprunte'].values, taux, duree)
    dff['Cashflow_Net'] = dff['Loyer_Mensuel_Estime'] - dff['Mensualite']

    nb_total = len(dff)
    nb_pos = int((dff['Cashflow_Net'] >= 0).sum())

    with calc_output:
        clear_output(wait=True)
        print(f"Budget projet :       {budget:,.0f} €")
        print(f"Apport personnel :     {apport:,.0f} €")
        print(f"Taux nominal :           {taux:.2f} %")
        print(f"Durée :                  {duree} ans")
        print("—" * 42)
        print(f"Biens affichés : {nb_total} | Cash-flow ≥ 0 : {nb_pos} ({nb_pos/nb_total:.0%})")

    # Palette plus expressive (Violet -> Rouge -> Orange -> Vert)
    custom_colors = [
        (0.00, "#4B0082"),  # violet profond
        (0.25, "#8B0000"),  # rouge foncé
        (0.45, "#FF8C00"),  # orange (juste sous 0)
        (0.55, "#FFFF99"),  # jaune clair (autour de 0)
        (0.75, "#7FFF00"),  # vert clair
        (1.00, "#006400")   # vert foncé
    ]

    # Étendue de couleur étendue : -2000 à +2000 €/mois
    cmin, cmax = -2000, 2000

    fig = px.scatter_mapbox(
        dff,
        lat="latitude", lon="longitude",
        size="surface_reelle_bati", size_max=20,
        color="Cashflow_Net",
        color_continuous_scale=custom_colors,
        range_color=[cmin, cmax],
        hover_name="adresse_nom_voie",
        center={"lat": dff['latitude'].mean(), "lon": dff['longitude'].mean()},
        zoom=11,
        mapbox_style="open-street-map",
        height=650,
        title=f"DVF + Loyers — Cash-flow net mensuel (€/mois)"
    )

    fig.update_layout(
        margin=dict(l=0, r=0, t=60, b=0),
        coloraxis_colorbar=dict(
            title="Cash-flow net (€/mois)",
            tickvals=[-2000,-1000,0,1000,2000],
            ticktext=["< -2000","-1000","0","+1000","> +2000"]
        )
    )

    gofig = go.FigureWidget(fig)

    def on_point_click(trace, points, state):
        with click_output:
            clear_output(wait=True)
            if not points.point_inds:
                return
            i = points.point_inds[0]
            row = dff.iloc[i]
            print("—— Fiche bien ——")
            print(f"Adresse : {row.get('adresse_nom_voie','N/A')}, {row.get('nom_commune','N/A')}")
            print(f"Prix : {row['valeur_fonciere']:,.0f} €  |  Surface : {row['surface_reelle_bati']:,.1f} m²")
            print(f"Loyer estimé : {row['Loyer_Mensuel_Estime']:,.0f} €/mois")
            print(f"Mensualité : {row['Mensualite']:,.0f} €/mois")
            print(f"Cash-flow net : {row['Cashflow_Net']:,.0f} €/mois")

    if len(gofig.data):
        gofig.data[0].on_click(on_point_click)

    with map_output:
        clear_output(wait=True)
        display(gofig)


permet de relancer les fonctions quand les sliders bouge

In [ ]:
# ==================== Wiring ====================
for w in (apport_slider, budget_slider, taux_slider, duree_slider):
    w.observe(render, names='value')

affichage 

In [ ]:
display(budget_slider)
display(apport_slider)
display(taux_slider)
display(duree_slider)
display(calc_output)
display(click_output)
display(map_output)

render()

IntSlider(value=300000, description='Budget (€):', layout=Layout(width='80%'), max=1000000, step=10000, style=…

IntSlider(value=50000, description='Apport (€):', layout=Layout(width='80%'), max=170000, step=5000, style=Sli…

FloatSlider(value=3.5, description='Taux prêt (%):', layout=Layout(width='80%'), max=15.0, style=SliderStyle(d…

IntSlider(value=25, description='Durée (années):', layout=Layout(width='80%'), max=30, min=5, style=SliderStyl…

Output()

Output()

Output()